In [1]:
import os.path
has_train_data = os.path.isfile('train.p') 
if not has_train_data:
    from zipfile import ZipFile
    import urllib.request
    from tempfile import mktemp

    zipurl = 'https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581faac4_traffic-signs-data/traffic-signs-data.zip'
    from urllib.request import urlopen
    from zipfile import ZipFile
    zipresp = urlopen(zipurl)
    tempzip = open("/tmp/tempfile.zip", "wb")
    tempzip.write(zipresp.read())
    tempzip.close()
    zf = ZipFile("/tmp/tempfile.zip")
    zf.extractall()
    zf.close()

In [2]:
import pickle
train_data = 'train.p'
test_data = 'test.p'
with open(train_data, 'rb') as f:
    pickle_data = pickle.load(f)
    train_features = pickle_data['features']
    train_labels = pickle_data['labels']
    del pickle_data  # Free up memory
with open(test_data, 'rb') as f:
    pickle_data = pickle.load(f)
    test_features = pickle_data['features']
    test_labels = pickle_data['labels']
    del pickle_data  # Free up memory


In [3]:
import cv2
import numpy as np
gray_train_features = [np.array(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY),dtype=np.float32).flatten() for image in  train_features]
gray_test_features = [np.array(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY),dtype=np.float32).flatten() for image in  test_features]

In [4]:
def normalize_greyscale(image, a=0.1, b=0.9,grey_min=0,grey_max=255):
    return a + ( ( (image - grey_min)*(b - a) )/( grey_max - grey_min ) )
train_features = [normalize_greyscale(image) for image in gray_train_features]
test_features = [normalize_greyscale(image) for image in gray_test_features]

In [5]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
encoder.fit(train_labels)
train_labels = encoder.transform(train_labels)
test_labels = encoder.transform(test_labels)

In [6]:
from sklearn.utils import shuffle
train_features, train_labels = shuffle(train_features, train_labels)

In [7]:
# Change to float32, so it can be multiplied against the features in TensorFlow, which are float32
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

In [8]:
import tensorflow as tf

# Parameters
learning_rate = 0.01
training_epochs = 100
batch_size = 50
display_step = 1



# tf Graph Input
x = tf.placeholder(tf.float32, [None, 1024]) # image of shape 32*32=1024
y = tf.placeholder(tf.float32, [None, 43]) # 43 classes

# Set model weights
W = tf.Variable(tf.zeros([1024, 43]))
b = tf.Variable(tf.zeros([43]))

pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [9]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(train_features)/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_start = i*batch_size
            batch_features = train_features[batch_start:batch_start + batch_size]
            batch_labels = train_labels[batch_start:batch_start + batch_size]
            
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_features, y: batch_labels})
            
            # Compute average loss
            avg_cost += c / total_batch
            
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy on test data:", accuracy.eval({x: test_features, y: test_labels}))
    print("Accuracy on training data:", accuracy.eval({x: train_features, y: train_labels}))

Epoch: 0001 cost= 3.337981052
Epoch: 0002 cost= 3.061577649
Epoch: 0003 cost= 2.869748512
Epoch: 0004 cost= 2.722291455
Epoch: 0005 cost= 2.603159435
Epoch: 0006 cost= 2.503220857
Epoch: 0007 cost= 2.417062013
Epoch: 0008 cost= 2.341296130
Epoch: 0009 cost= 2.273683978
Epoch: 0010 cost= 2.212670266
Epoch: 0011 cost= 2.157127867
Epoch: 0012 cost= 2.106209807
Epoch: 0013 cost= 2.059260179
Epoch: 0014 cost= 2.015758291
Epoch: 0015 cost= 1.975282274
Epoch: 0016 cost= 1.937484513
Epoch: 0017 cost= 1.902074352
Epoch: 0018 cost= 1.868805727
Epoch: 0019 cost= 1.837468043
Epoch: 0020 cost= 1.807879406
Epoch: 0021 cost= 1.779881487
Epoch: 0022 cost= 1.753335585
Epoch: 0023 cost= 1.728119377
Epoch: 0024 cost= 1.704124596
Epoch: 0025 cost= 1.681254874
Epoch: 0026 cost= 1.659424215
Epoch: 0027 cost= 1.638555417
Epoch: 0028 cost= 1.618579145
Epoch: 0029 cost= 1.599432869
Epoch: 0030 cost= 1.581060019
Epoch: 0031 cost= 1.563409231
Epoch: 0032 cost= 1.546433821
Epoch: 0033 cost= 1.530091236
Epoch: 003